# Практика
✍ В этом модуле мы обсудили, как используются рекомендательные системы, как получают данные для них и как оценивают результаты работы РС. Мы начали знакомиться с алгоритмами построения систем рекомендаций и пока успели изучить более подробно один из них — РС на основе популярности. В этом юните мы построим систему рекомендаций, основываясь именно на этом методе. Вы сможете усовершенствовать её в следующем модуле, после того как освоите другие алгоритмы.

Для начала загрузим датасет "Articles sharing and reading from CI&T DeskDrop", включающий в себя собранные за один год логи DeskDrop — платформы для внутренних коммуникаций, разработанной CI&T и ориентированной на компании, использующие Google Workspace (Google G Suite). Среди прочего, эта платформа позволяет сотрудникам компаний делиться актуальными статьями со своими коллегами.

В датасете содержится около 73 тысяч записей о взаимодействии пользователей с более чем тремя тысячами публичных статей, размещённых на платформе.

Информация в наборе данных:
Оригинальный URL, название и текст статьи.
Контекст посещений пользователей, например дата/время, клиент (мобильное приложение/браузер) и геолокация.
Различные типы взаимодействия, что позволяет сделать вывод об уровне заинтересованности пользователя в статьях, например комментарии → лайки → просмотры.
Данные включают в себя два файла:

shared_articles.csv;
users_interactions.csv.
Начнём работать с файлом shared_articles.csv. Он содержит информацию о статьях, опубликованных на платформе DeskDrop.

Для каждой статьи есть:
дата публикации (временная метка),
исходный URL-адрес,
заголовок,
содержание в виде обычного текста,
язык статьи (португальский — pt или английский — en),
информация о пользователе, который поделился статьёй (автор).
Для временной метки существует два возможных типа событий:

CONTENT SHARED — статья была опубликована на платформе и доступна для пользователей;
CONTENT REMOVED — статья была удалена с платформы и недоступна для дальнейших рекомендаций.
Для простоты мы рассматриваем здесь только тип события CONTENT SHARED.

In [1]:
import pandas as pd

In [2]:
shared_articles = pd.read_csv('/Users/egor/Documents/data_science_course/SKILLFACTORY/MATH&ML-14. Рекомендательные системы. Часть I/data/shared_articles.csv')
users_interactions = pd.read_csv('/Users/egor/Documents/data_science_course/SKILLFACTORY/MATH&ML-14. Рекомендательные системы. Часть I/data/users_interactions.csv')

Задание 6.1

Отфильтруйте данные так, чтобы остались только объекты с типом события CONTENT SHARED. Сколько таких объектов в получившейся таблице?

In [6]:
shared = shared_articles[shared_articles['eventType'] == 'CONTENT SHARED']
shared.shape[0]

3047

Теперь откроем второй файл — users_interactions.csv.

Давайте предварительно преобразуем столбцы personId, contentId в таблицах к строкам. Это преобразование пригодится нам в дальнейшем:

In [7]:
users_interactions.personId = users_interactions.personId.astype(str)
users_interactions.contentId = users_interactions.contentId.astype(str)
shared.contentId = shared.contentId.astype(str)

/var/folders/fg/7cyb3m_113j_1hkfwhtbsy8h0000gn/T/ipykernel_10772/778304297.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shared.contentId = shared.contentId.astype(str)


В колонке eventType описаны действия, которые могли совершать пользователи при взаимодействии со статьёй:

VIEW — просмотр,
LIKE — лайк,
COMMENT CREATED — комментарий,
FOLLOW — подписка,
BOOKMARK — добавление в закладки.
В первую очередь нам необходимо понять, как определить, что какая-то статья популярнее других. Если бы из возможных реакций у нас были только лайки или только просмотры, то статьи было бы легко ранжировать в соответствии с этими значениями. Однако у нас есть информация о различных действиях пользователя, и на её основе мы должны создать некий универсальный индекс популярности. Составим его из реакций пользователей, придав им разные веса:

In [9]:
users_interactionsevent_type = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

Веса здесь подобраны исходя из важности каждого действия: оставить комментарий — значит, показать наибольшую вовлечённость, а обычный просмотр, напротив, демонстрирует наименьшую вовлечённость.

Задание 6.2

Создайте признак, который будет отражать числовой вес для взаимодействия со статьёй (в соответствии с приведёнными выше весами). Вычислите среднее значение для полученного признака. Округлите его до двух знаков после точки-разделителя.

In [18]:
users_interactions['score'] = users_interactions['eventType'].apply(lambda x: users_interactionsevent_type[x])
users_interactions['score'].mean()

1.2362885828078327

Ранее мы говорили, что рекомендательные системы подвержены проблеме холодного старта — в таких случаях создавать рекомендации намного сложнее.

Задание 6.3

Чтобы получить хоть какую-то информацию, на которую можно будет опираться, оставьте только тех пользователей, которые взаимодействовали хотя бы с пятью статьями. Сколько всего таких пользователей?

In [34]:
int_count = users_interactions.groupby(by='personId')['contentId'].nunique()
int_count[int_count > 4].shape[0]

1140

Задание 6.4

Теперь оставим только те взаимодействия, которые касаются только отфильтрованных пользователей (то есть тех, которые взаимодействовали как минимум с пятью статьями). Сколько всего таких взаимодействий?



In [40]:
interactions = users_interactions[
    users_interactions['personId'].isin(int_count[int_count > 4].index)
    ]
interactions.shape[0]

69868

Сейчас каждое отдельное взаимодействие пользователя со статьёй выделено в отдельную запись, то есть пользователь мог просмотреть статью, лайкнуть и прокомментировать её, и всё это отразилось в трёх действиях. Давайте для удобства соединим все эти действия в некоторый коэффициент, который будет отражать интерес пользователя к статье. Так как каждому возможному действию мы ранее уже присвоили вес, то, по сути, нам нужно просто сложить все действия. Однако полученное число будет увеличиваться с количеством действий, и будет очень большой разброс возможных значений. В таких случаях обычно логарифмируют полученный результат с помощью следующей функции:

In [45]:
import math
def smooth_user_preference(x):
    return math.log(1+x, 2)

Задание 6.5

Примените упомянутое выше преобразование для логарифмирования к сумме весов для взаимодействия пользователя с каждой конкретной статьёй. Также сохраните для каждой пары «пользователь — статья» значение времени последнего взаимодействия.

Найдите среднее по признаку с получившимися временными отсечками. Округлите результат до двух знаков после точки-разделителя.

Так как наши данные отсортированы по дате, то для того, чтобы выбрать последнее взаимодействие, необходимо использовать метод max().

In [56]:
person_content = interactions.groupby(
    by=['personId', 'contentId'], as_index=False
    )[['eventType', 'score', 'timestamp']].agg(
        {'eventType': 'nunique',
        'score': 'sum',
        'timestamp': 'max'})

In [57]:
person_content['score'] = person_content['score'].apply(lambda x: smooth_user_preference(x))

In [60]:
person_content['timestamp'].mean()

1470605340.0403006

In [61]:
person_content

,personId,contentId,eventType,score,timestamp
0,-1007001694607905623,-5065077552540450930,1,1.000000,1470395911
1,-1007001694607905623,-6623581327558800021,1,1.000000,1487240080
2,-1007001694607905623,-793729620925729327,1,1.000000,1472834892
3,-1007001694607905623,1469580151036142903,1,1.000000,1487240062
4,-1007001694607905623,7270966256391553686,1,1.584963,1485994342
...,...,...,...,...,...
39101,998688566268269815,-401664538366009049,1,1.000000,1474567449
39102,998688566268269815,3456674717452933449,1,2.584963,1478802088
39103,998688566268269815,6881796783400625893,1,1.000000,1474567675
39104,998688566268269815,7174452660053929140,1,2.321928,1478812905


Разумеется, для того чтобы впоследствии оценить качество построенной рекомендательной системы, нам нужно разделить выборку на обучающую и тестовую. Так как в реальности рекомендации строятся на основе исторических данных о пользователе и контенте, сделаем в нашей задаче разбиение на обучающую и тестовую выборки по временной отсечке.

Задание 6.6

Разделите данные на обучающую и тестовую выборки, выбрав в качестве временной отсечки значение 1475519545. Значение отсечки включите в тестовую выборку. Сколько объектов попало в обучающую выборку?

In [65]:
train_df = person_content[person_content['timestamp'] < 1475519545]
test_df = person_content[person_content['timestamp'] >= 1475519545]
train_df.shape[0]

29325

Для удобства дальнейшего измерения качества рекомендаций преобразуйте данные так, чтобы получить таблицу в формате, где строка соответствует пользователю, а столбцы будут истинными предпочтениями и рекомендациями в формате списков. На место пустых ячеек поместите пустые списки.

In [66]:
import numpy as np

final_df = (
    train_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

final_df['true_test'] = (
    test_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

final_df['true_test'] = [ [] if x is np.NaN else x for x in final_df['true_test'] ]
final_df.head()

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."
-1032019229384696495,"[-1006791494035379303, -1039912738963181810, -...","[-1415040208471067980, -2555801390963402198, -..."
-108842214936804958,"[-1196068832249300490, -133139342397538859, -1...","[-2780168264183400543, -3060116862184714437, -..."
-1130272294246983140,"[-1150591229250318592, -1196068832249300490, -...","[-1606980109000976010, -1663441888197894674, -..."
-1160159014793528221,"[-133139342397538859, -387651900461462767, 377...",[-3462051751080362224]


Задание 6.7

Осталось совсем немного — скоро вы получите свою первую систему рекомендаций! Мы будем строить popular-based-модель, а значит, нам необходимо найти самые популярные статьи.

Посчитайте популярность каждой статьи как сумму всех логарифмических «оценок» взаимодействий с ней (используя только обучающую выборку). Выберите ID самой популярной статьи:

In [80]:
train_df.groupby(by='contentId')['score'].sum().sort_values(ascending=False).head(10)

contentId
-6783772548752091658    231.177195
-133139342397538859     228.024567
-8208801367848627943    189.937683
8224860111193157980     186.044680
7507067965574797372     179.094002
-2358756719610361882    175.771101
-6843047699859121724    175.108147
-1297580205670251233    160.671086
8657408509986329668     157.973460
3367026768872537336     149.383615
Name: score, dtype: float64

Теперь необходимо сформировать рекомендации для каждого пользователя. Будем рекомендовать десять самых популярных статей. Также необходимо помнить, что следует предлагать пользователю только то, что он ещё не читал.

Задание 6.8

Постройте систему рекомендаций. Оцените качество с помощью precision@10 для каждого пользователя (доля угаданных рекомендаций). После этого усредните результат по всем пользователям.

Для вычисления precision@10 воспользуйтесь следующей функцией:

In [71]:
def precision(column):
    return (final_df.apply(lambda row:
            len(set(row['true_test']).intersection(set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),axis=1)).mean()

In [107]:
top_popular = train_df.groupby(by='contentId')['score'].sum().sort_values(ascending=False)
top_popular = top_popular.index

In [109]:
import numpy as np

top_k = 10

final_df['popular'] = final_df['true_train'].apply(lambda x: top_popular[~np.in1d(top_popular, x)][:top_k])

In [111]:
precision('popular')

0.006454207722621089